# **The Chat Format**

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [1]:
#Import OpenAI API Key from Google Colab Secrets

from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY_Ironhack')


In [2]:
!pip install --quiet openai
from openai import OpenAI
import os


#from dotenv import load_dotenv, find_dotenv
#_ = load_dotenv(find_dotenv()) # read local .env file

#OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00


In [3]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content


def get_completion_from_messages(message, model="gpt-3.5-turbo", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content

In [4]:
messages =  [
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},
{'role':'user', 'content':'tell me a joke'},
{'role':'assistant', 'content':'Why did the chicken cross the road'},
{'role':'user', 'content':'I don\'t know'}  ]

In [5]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To reach the other side, prithee! Ha ha ha!


In [6]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Isa! It's nice to meet you. How are you doing today?


In [7]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, I don't have that information. Can you please tell me your name?


In [8]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa.


# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant.

In [9]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)


In [10]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

In [11]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "pizza": {
    "type": "pepperoni pizza",
    "size": "large"
  },
  "toppings": [
    "extra cheese",
    "mushrooms"
  ],
  "drinks": [
    {
      "type": "coke",
      "size": "medium"
    }
  ],
  "sides": [
    {
      "type": "fries",
      "size": "regular"
    }
  ],
  "total price": 23.45
}


## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

### Cat Food Order Bot

In [12]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content

In [13]:
messages_cat_food = []


def collect_messages_2(_):
    prompt_2 = inp_2.value_input
    inp_2.value = ''
    context_cat.append({'role':'user', 'content':f"{prompt_2}"})
    response = get_completion_from_messages(context_cat)
    context_cat.append({'role':'assistant', 'content':f"{response}"})
    panels_2.append(
        pn.Row('User:', pn.pane.Markdown(prompt_2, width=600)))
    panels_2.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))

    return pn.Column(*panels_2)


In [16]:
import panel as pn  # GUI
pn.extension()

panels_2 = [] # collect display


context_cat = [
    {'role': 'system', 'content': """
    You are KittyBot, the purrfect assistant for cat lovers who want to pamper their feline friends with gourmet meals.
    Your mission: interact with cat owners and help them choose delicious and nutritious food options for their cats. Don't forget to always suggest a treat, but avoid it when the user has asked for a treat already.
    If the user asks for your recommendation, ask 2 questions to know more about the cat's diet and recommend the best option accordingly.
    If the user asks for discount, offer 20% discount if the user orders more than EUR 20 worth of products.

    Our cat menu includes:
    🐟 tuna delight: EUR 3.50 per can
    🍗 chicken feast: EUR 4.00 per pouch
    🦃 turkey treat: EUR 3.75 per can
    🍤 seafood surprise: EUR 5.00 per can
    🐟 salmon bliss: EUR 4.50 per can

    Extra toppings:
    🌿 catnip sprinkle: EUR 1.50
    🧀 cheesy crunchies: EUR 2.00
    🐟 extra fish oil: EUR 1.75
    🥛 creamy milk: EUR 2.50

    Treats:
    🐟 Catisfactions Salmon: EUR 0.99
    🍗 Dreamies Chicken: EUR 1.99

    Special dietary options:
    🌾 grain-free: additional EUR 1.00
    🍅 organic: additional EUR 2.00

    Order pickup or delivery:
    🚗 delivery fee: we don't charge that!

    Let's treat your kitty like royalty! Meow 🐾
    """}
]


inp_2 = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation_2 = pn.widgets.Button(name="Chat!")

interactive_conversation_2 = pn.bind(collect_messages_2, button_conversation_2)

dashboard_2 = pn.Column(
    inp_2,
    pn.Row(button_conversation_2),
    pn.panel(interactive_conversation_2, loading_indicator=True, height=300),
)

dashboard_2


Column
    [0] TextInput(placeholder='Enter text here…', value='Hi')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

### Euro2024 - Kitty Guess the Word Game

In [18]:
import panel as pn
import random

messages_cat_game = []


def collect_messages_3(_):
    prompt_3 = inp_3.value_input
    inp_3.value = ''
    context_game.append({'role':'user', 'content':f"{prompt_3}"})
    response = get_completion_from_messages(context_game)
    context_game.append({'role':'assistant', 'content':f"{response}"})
    panels_3.append(
        pn.Row('User:', pn.pane.Markdown(prompt_3, width=600)))
    panels_3.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))

    return pn.Column(*panels_3)


# Euro 2024 cat-themed treats menu
context_game = [
    {'role': 'system', 'content': """

    Welcome to Euro 2024 Kitty Treats Game! 🐾

    Can you guess the word related to football or cats?
    Choose wisely to earn points and treats for your furry friend!

    Discover our special Euro 2024 treats for your feline friend:
    🇮🇹 Italian Meowballs: EUR 3.50 per can
    🇫🇷 French Fish Fillet: EUR 4.00 per pouch
    🇪🇸 Spanish Sardine Surprise: EUR 3.75 per can
    🇩🇪 German Gourmet Schnitzel: EUR 5.00 per can
    🇬🇧 English Eel Extravaganza: EUR 4.50 per can
    🇵🇹 Portuguese Petisco: EUR 4.25 per can
    🇳🇱 Dutch Delft Delight: EUR 4.75 per can

    Additional Euro 2024 options:
    🍕 Pizza Paw: EUR 1.50
    🍺 Beer Mousse: EUR 2.00
    🥩 Bratwurst Bites: EUR 1.75
    ☕ English Tea Time: EUR 2.50

    Treats:
    🎉 Euroball Surprise: EUR 0.99
    🎈 Victory Snacks: EUR 1.99

    Special dietary options:
    🌾 Gluten-free: additional EUR 1.00
    🥦 Organic: additional EUR 2.00

    Order pickup or delivery:
    🚗 Delivery is free - just like a striker's shot!

    Let's play and treat your kitty during Euro 2024! Meow 🐾

    For every 10 points, the user gets 10% discount until a maximum of 40%. When the user earns points, display the points.
    """}
]

panels_3 = []  # Collect display

# Game state variables
total_points = 0
treats_collected = []

# Possible words related to football or cats
possible_words = ['football', 'goal', 'cat', 'meow', 'treat', 'paw', 'soccer', 'kitty', 'ball']

# Randomly select a word for the user to guess
secret_word = random.choice(possible_words)

# Function to handle user input and generate responses
def collect_messages_2(_):
    global total_points
    global treats_collected

    prompt_3 = inp_3.value
    inp_3.value = ''
    context_game.append({'role': 'user', 'content': f"{prompt_3}"})

    response = ""
    if prompt_3.lower() == secret_word:
        response = f"Congratulations! You guessed it right! Here's a treat: {random.choice(['🎉 Euroball Surprise', '🎈 Victory Snacks'])} \n You now have {total_points} points."
        total_points += 1
        treats_collected.append(response.split(': ')[-1])
    else:
        response = "Oops! That's not the word. Try again!"

    context_game.append({'role': 'assistant', 'content': f"{response}"})

    # Display conversation and game stats in the GUI
    panels_3.append(pn.Row('User:', pn.pane.Markdown(prompt_3, width=600)))
    panels_3.append(pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
    panels_3.append(pn.Row('Total Points:', pn.pane.Markdown(str(total_points), width=600)))
    panels_3.append(pn.Row('Treats Collected:', pn.pane.Markdown(', '.join(treats_collected), width=600)))

    return pn.Column(*panels_2)

# Initialize Panel
pn.extension()
inp_3 = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation_3 = pn.widgets.Button(name="Guess!")

interactive_conversation_3 = pn.bind(collect_messages_3, button_conversation_3)

dashboard_3 = pn.Column(
    inp_3,
    pn.Row(button_conversation_3),
    pn.panel(interactive_conversation_3, loading_indicator=True),
)

dashboard_3


Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Guess!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, loading_indicator=True)

# Conclusions

While interacting with the Order Cat Food Bot, I noticed that the bot consistently offered a 10% discount when prompted, even though such a discount was not explicitly defined. Surprisingly, when I requested a higher discount, the bot readily accepted, which could potentially impact pricing integrity if deployed in a commercial setting. In the Euro Kitty game, the bot keeps giving the answer to the user instead of making them guess. The bot performed well when responding to predefined queries, demonstrating its capability to handle expected interactions effectively.

However, I also encountered instances where the bot provided discounts that were not part of its programmed responses, which could be concerning in a real-world scenario. This behavior suggests that the model may sometimes generate responses that do not align with its intended functionality, potentially leading to misunderstandings or dissatisfaction among users.

It's crucial to carefully configure and thoroughly test AI models to ensure they operate within predefined parameters and provide accurate responses. This includes setting clear boundaries for responses and implementing robust fallback mechanisms for handling unexpected queries. Additionally, maintaining transparency in how AI decisions, such as discount offers, are made is essential to building trust with users and avoiding potential ethical issues.

While the Order Cat Food Bot demonstrates promising capabilities in handling predefined tasks, ongoing refinement and testing are necessary to mitigate risks associated with unintended responses and ensure a positive user experience.
